In [3]:
from lxml import etree

annotations_path = "../../sources/Clear/annotations/annotations.xml" 
annotations_etree = etree.parse(annotations_path)
annotations_root = annotations_etree.getroot()

mappings_path = "../../mappings/morpheme-mappings.xml"
mappings_etree = etree.parse(mappings_path)
mappings_root = mappings_etree.getroot()

In [4]:
nodes = annotations_root.xpath(f"//node[@Frame or @SubjRef or @Ref]")

In [122]:
def replace_id(morphId):
    return mappings_dict_morphId.get(morphId, {"oshb": morphId})["oshb"]

In [144]:
count

15656

In [146]:
def convert_frame(frame):
    return "{" + " ".join([
        a[:3] + ";".join([replace_id(morph_id) for morph_id in a[3:].split(";")])
        for a in frame[0][1:-1].split(" ")
    ]) + "}"

In [147]:
def convert_ref(ref):
    return "{" + " ".join([replace_id(morph_id)
                           for morph_id in ref[0][1:-1].split(" ")]) + "}"

In [148]:
count = 0
for n in nodes:
    frame = n.xpath(f"@Frame")
    subjref = n.xpath(f"@SubjRef")
    ref = n.xpath(f"@Ref")
    
    if subjref:
        n.attrib["SubjRef"] = convert_ref(subjref)
        if not subjref[0] == convert_ref(subjref):
            count += 1
    if frame:
        n.attrib["Frame"] = convert_frame(frame)
        if not frame[0] == convert_frame(frame):
            count += 1
    if ref:
        n.attrib["Ref"] = convert_ref(ref)
        if not ref[0] == convert_ref(ref):
            count += 1

In [149]:
count

6091

In [105]:
ref_set = {i for ref in annotations_root.xpath(f"//@Ref") for i in ref[1:-1].split(" ")}
subjref_set = {i for ref in annotations_root.xpath(f"//@SubjRef") for i in ref[1:-1].split(" ")}
frame_set = {
    k
    for n in annotations_root.xpath(f"//node[@Frame]")
    for k in [
        i for ids in n.attrib["Frame"][1:-1].split(" ")
        for i in ids[3:].split(";") if i
    ] 
}
mapping_morph_set = set(mappings_root.xpath(f"//m/@morphId"))
grooves_set = ref_set | subjref_set | frame_set

print(len(grooves_set) - len(morph_set & grooves_set))

54


In [110]:
for m in mappings_root:
    etree.dump(m)
    break

<m n="010010010011" marble="00100100100002" morphId="010010010011"/>
  


In [116]:
mappings_dict_oshb = {
    m.attrib["n"]: {
        "marble": "" if not "marble" in m.attrib else m.attrib["marble"],
        "morphId": "" if not "morph" in m.attrib else m.attrib["morphId"]
    }
    for m in mappings_root
}

In [115]:
mappings_dict_morphId = {
    m.attrib["morphId"]: {
        "oshb": m.attrib["n"],
        "marble": "" if not "marble" in m.attrib else m.attrib["marble"]
    }
    for m in mappings_root if "morphId" in m.attrib
}

In [134]:
a = sorted(grooves_set - morph_set)

In [136]:
for i in a:
    if i in mappings_dict_morphId:
        print(i)